In [24]:
import pandas as pd

meld_dir = "meld_dataset"
split = "train"
dialogue_file = os.path.join(meld_dir, f'meld_{split}.csv')
meld_df = pd.read_csv(dialogue_file)

ross_dialogues = meld_df[meld_df['Speaker'] == 'Ross']

ross_neutral_dialogues = ross_dialogues[ross_dialogues['Emotion'] == 'neutral']

print("Number of neutral dialogues involving Ross:", len(ross_neutral_dialogues))

print("\nSample of neutral dialogues involving Ross:")
print(ross_neutral_dialogues.head())

Number of neutral dialogues involving Ross: 719

Sample of neutral dialogues involving Ross:
    Sr No.                                          Utterance Speaker  \
46      50  Ah y'know, this building is on my paper route ...    Ross   
48      52                                                Hi.    Ross   
50      54                                   Howd did it go?    Ross   
56      60                                              Yeah.    Ross   
97     101                                            I know.    Ross   

    Emotion Sentiment  Dialogue_ID  Utterance_ID  Season  Episode  \
46  neutral   neutral            4             3       3       11   
48  neutral   neutral            4             5       3       11   
50  neutral   neutral            4             7       3       11   
56  neutral   neutral            4            13       3       11   
97  neutral   neutral            9             4       2        5   

       StartTime       EndTime  
46  00:20:26,433  00

In [35]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchaudio import transforms as T
from torchvision.io import read_video

class MELDDataset(Dataset):
    def __init__(self, meld_dir, split, transform=None):
        self.meld_dir = meld_dir
        self.split = split
        self.transform = transform
        
        
        self.dialogues = self.load_dialogues()
        
    def load_dialogues(self):
        dialogue_file = os.path.join(self.meld_dir, f'meld_{self.split}.csv')
        dialogues = pd.read_csv(dialogue_file)
        return dialogues

    def __len__(self):
        return len(self.dialogues)

    def __getitem__(self, idx):
        row = self.dialogues.iloc[idx]
        
        
        text = row['Utterance']
        video_dir = os.path.join(self.meld_dir, 'wav', f'audio_files_for_{self.split}_set')
        video_file = os.path.join(video_dir, f'dia{row["Dialogue_ID"]}_utt{row["Utterance_ID"]}.mp4')
        label = row['Emotion']
        
        try:
            
            video_data, audio_data, info = read_video(video_file, pts_unit='sec')
        except Exception as e:
            print(f"Error loading video file: {str(e)}")
            return None, None, None
        
        
        audio_data = audio_data[:, 0]  
        
        
        if self.transform:
            
            audio_data = self.transform(audio_data)

        return text, audio_data, label